<a href="https://colab.research.google.com/github/amhaiskar0921/BTTAIAmazonProject/blob/main/FinalComplementNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Complement Naive Bayes for Imbalaced Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score, classification_report

In [4]:
df_merged = pd.read_csv('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/balanced_preprocessed_merged_data.csv')
df_merged.head()

,product_id,product_locale,product_title,product_description,product_brand,esci_label,query,bert_input
0,B0743Z7W7L,us,megaman retro arcad 8 bite decal mega man stic...,mega man retro 8 bite decal easi appli vehicl ...,decalcomania llc,C,bass action figur megaman,NaN
1,B076SZ62X2,us,hikig car headrest mount holder kid kindl fire...,hikig car headrest mount holder kid kindl fire...,hikig,C,tablet car,NaN
2,B07JYZ55DQ,es,lijumn performance etiqueta engomado fibra car...,NaN,lijumn,C,bmw x3 pegatín llave,NaN
3,B07YD9M3YW,es,jztrading alfombrar titular baño tira resisten...,color neromateriale osito gomadimensión prodot...,jztrading,C,alfombra cocina,NaN
4,B00H6Z4LI4,jp,kindle paperwhite 専用 小型 usb 充電 器 505 0003,NaN,和 湘 堂 washodo,C,whitepaper kindle 広告 なし,NaN


In [26]:
print(len(df_merged))

100868


In [5]:
df_merged.drop(columns=["bert_input", "product_id"], inplace=True)

In [6]:
df_merged.fillna("", inplace=True)
df_merged.head()

,product_locale,product_title,product_description,product_brand,esci_label,query
0,us,megaman retro arcad 8 bite decal mega man stic...,mega man retro 8 bite decal easi appli vehicl ...,decalcomania llc,C,bass action figur megaman
1,us,hikig car headrest mount holder kid kindl fire...,hikig car headrest mount holder kid kindl fire...,hikig,C,tablet car
2,es,lijumn performance etiqueta engomado fibra car...,,lijumn,C,bmw x3 pegatín llave
3,es,jztrading alfombrar titular baño tira resisten...,color neromateriale osito gomadimensión prodot...,jztrading,C,alfombra cocina
4,jp,kindle paperwhite 専用 小型 usb 充電 器 505 0003,,和 湘 堂 washodo,C,whitepaper kindle 広告 なし


##Data Prep

###Step 1: Combining relevant columns to create input column

In [7]:
# Create a new column for BERT input
df_merged['nb_input'] = ""

# Apply functions based on the language specified in the product_locale column
for index, row in df_merged.iterrows():
    locale = row['product_locale']
    query_text = row['query']
    title_text = row['product_title']
    description_text = row['product_description']
    brand_text = row['product_brand']


    nb_input = f'{query_text} {title_text} {description_text} {brand_text}'
    df_merged.at[index, 'nb_input'] = nb_input

###Step 2: Numerical mapping of label column

In [8]:
label_map = {"E": 0, "S": 1, "C": 2, "I": 3}

# Assuming df_merged is your DataFrame containing the "esci_label" column
df_merged["esci_label"] = df_merged["esci_label"].map(label_map)

In [9]:
df_merged.head()

,product_locale,product_title,product_description,product_brand,esci_label,query,nb_input
0,us,megaman retro arcad 8 bite decal mega man stic...,mega man retro 8 bite decal easi appli vehicl ...,decalcomania llc,2,bass action figur megaman,bass action figur megaman megaman retro arcad ...
1,us,hikig car headrest mount holder kid kindl fire...,hikig car headrest mount holder kid kindl fire...,hikig,2,tablet car,tablet car hikig car headrest mount holder kid...
2,es,lijumn performance etiqueta engomado fibra car...,,lijumn,2,bmw x3 pegatín llave,bmw x3 pegatín llave lijumn performance etique...
3,es,jztrading alfombrar titular baño tira resisten...,color neromateriale osito gomadimensión prodot...,jztrading,2,alfombra cocina,alfombra cocina jztrading alfombrar titular ba...
4,jp,kindle paperwhite 専用 小型 usb 充電 器 505 0003,,和 湘 堂 washodo,2,whitepaper kindle 広告 なし,whitepaper kindle 広告 なし kindle paperwhite 専用 小...


##One-hot encoding with TF-IDF and Count Vectorizer

In [10]:
y = df_merged['esci_label']
X = df_merged['nb_input']

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
X_test_count = scaler.fit_transform(X_test_count)

##Creating the model

###Training on TF-IDF data

In [14]:
# Create a Complement Naive Bayes classifier
cnb_model = ComplementNB()

In [15]:
# Train the model
cnb_model.fit(X_train_tfidf, y_train)

ComplementNB()

In [16]:
# Make predictions on the test set
y_pred_tfidf = cnb_model.predict(X_test_tfidf)

In [17]:
# Evaluate the best model using micro-averaging F1-score
micro_f1 = f1_score(y_test, y_pred_tfidf, average='micro')
print(f'Micro-Averaging F1-score (TF-IDF): {micro_f1}')

Micro-Averaging F1-score (TF-IDF): 0.41375037176563895


In [18]:
report = classification_report(y_test, y_pred_tfidf)
print('Classification Report:\n', report)

Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.48      0.41      5188
           1       0.38      0.25      0.30      5000
           2       0.48      0.61      0.54      5057
           3       0.46      0.31      0.37      4929

    accuracy                           0.41     20174
   macro avg       0.42      0.41      0.40     20174
weighted avg       0.42      0.41      0.40     20174



###Training on Count Vectorizer data

In [21]:
cnb_model = ComplementNB()

In [22]:
# Train the model
cnb_model.fit(X_train_count, y_train)

ComplementNB()

In [23]:
# Make predictions on the test set
y_pred_count = cnb_model.predict(X_test_count)

In [24]:
micro_f1 = f1_score(y_test, y_pred_count, average='micro')
print(f'Micro-Averaging F1-score (Count): {micro_f1}')

Micro-Averaging F1-score (Count): 0.40552195895707344


In [25]:
report = classification_report(y_test, y_pred_count)
print('Classification Report:\n', report)

Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.35      0.35      5188
           1       0.35      0.32      0.33      5000
           2       0.50      0.57      0.53      5057
           3       0.40      0.39      0.40      4929

    accuracy                           0.41     20174
   macro avg       0.40      0.41      0.40     20174
weighted avg       0.40      0.41      0.40     20174

